In [6]:
import os
import cv2
import numpy as np
from skimage import measure, morphology
import skfuzzy as fuzz
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.metrics import accuracy_score, jaccard_score
import pandas as pd

def load_images(directory, target_size=(256, 256)):
    images = []
    filenames = [f for f in os.listdir(directory) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for filename in tqdm(filenames, desc=f"Loading images from {directory}"):
        img = cv2.imread(os.path.join(directory, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, target_size)
            img = cv2.normalize(img, None, 0, 1, cv2.NORM_MINMAX).astype(np.float32)
            images.append(img)
    return np.array(images), filenames

def preprocess_image(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply((blurred*255).astype(np.uint8))
    return enhanced.astype(np.float32)/255.0

def fuzzy_cmeans_segmentation(image, n_clusters=3):
    pixel_values = image.flatten().reshape(1, -1)
    cntr, u, _, _, _, _, _ = fuzz.cluster.cmeans(
        pixel_values, n_clusters, 2, error=0.005, maxiter=1000
    )
    tumor_cluster = np.argmax(cntr)
    membership = u[tumor_cluster].reshape(image.shape)
    binary_mask = (membership > 0.5).astype(np.uint8)
    return binary_mask

def kmeans_segmentation(image, n_clusters=2):
    data = image.flatten().reshape(-1,1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10).fit(data)
    labels = kmeans.labels_.reshape(image.shape)
    tumor_cluster = np.argmax(kmeans.cluster_centers_)
    mask = (labels == tumor_cluster).astype(np.uint8)
    return mask

def post_process(mask, min_size=200):
    labels = measure.label(mask)
    if labels.max() == 0:
        return mask
    # keep only largest connected component
    areas = [np.sum(labels==i) for i in range(1, labels.max()+1)]
    largest_label = np.argmax(areas) + 1
    final_mask = (labels == largest_label).astype(np.uint8)
    # remove small noise
    final_mask = morphology.remove_small_objects(final_mask.astype(bool), min_size=min_size).astype(np.uint8)
    return final_mask

def evaluate_segmentation(true_mask, pred_mask):
    true_binary = (true_mask > 0).astype(np.uint8)
    pred_binary = (pred_mask > 0).astype(np.uint8)
    accuracy = accuracy_score(true_binary.flatten(), pred_binary.flatten())
    iou = jaccard_score(true_binary.flatten(), pred_binary.flatten())
    intersection = np.logical_and(true_binary, pred_binary)
    dice = (2. * intersection.sum()) / (true_binary.sum() + pred_binary.sum() + 1e-7)
    return accuracy, iou, dice

def main():
    TUMOR_DIR = r"C:\Users\hp\OneDrive\Desktop\yes"
    NORMAL_DIR = r"C:\Users\hp\OneDrive\Desktop\no"
    OUTPUT_DIR = "segmentation_results"
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    tumor_images, tumor_fnames = load_images(TUMOR_DIR)
    normal_images, normal_fnames = load_images(NORMAL_DIR)
    results = []

    print("\nProcessing tumor images...")
    for i, img in enumerate(tqdm(tumor_images)):
        processed = preprocess_image(img)
        fcm_mask = fuzzy_cmeans_segmentation(processed)
        k_mask = kmeans_segmentation(processed)
        combined_mask = np.logical_or(fcm_mask, k_mask).astype(np.uint8)
        final_mask = post_process(combined_mask)

        # Pseudo-ground truth for demo
        _, gt_mask = cv2.threshold((processed*255).astype(np.uint8), 0, 1, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        accuracy, iou, dice = evaluate_segmentation(gt_mask, final_mask)

        results.append({
            'filename': tumor_fnames[i],
            'type': 'tumor',
            'accuracy': accuracy,
            'iou': iou,
            'dice': dice
        })

        cv2.imwrite(os.path.join(OUTPUT_DIR, f"tumor_seg_{i}.png"), (final_mask*255).astype(np.uint8))

    print("\nProcessing normal images...")
    for i, img in enumerate(tqdm(normal_images)):
        processed = preprocess_image(img)
        fcm_mask = fuzzy_cmeans_segmentation(processed)
        k_mask = kmeans_segmentation(processed)
        combined_mask = np.logical_or(fcm_mask, k_mask).astype(np.uint8)
        final_mask = post_process(combined_mask)

        gt_mask = np.zeros_like(final_mask)

        accuracy, iou, dice = evaluate_segmentation(gt_mask, final_mask)

        results.append({
            'filename': normal_fnames[i],
            'type': 'normal',
            'accuracy': accuracy,
            'iou': iou,
            'dice': dice
        })

        cv2.imwrite(os.path.join(OUTPUT_DIR, f"normal_seg_{i}.png"), (final_mask*255).astype(np.uint8))

    df = pd.DataFrame(results)
    df.to_csv(os.path.join(OUTPUT_DIR, "segmentation_metrics.csv"), index=False)
    print("\nSegmentation metrics saved to", os.path.join(OUTPUT_DIR, "segmentation_metrics.csv"))

    print("\nAverage metrics:")
    print(df.groupby('type')[['accuracy','iou','dice']].mean())

if __name__ == "__main__":
    main()


Loading images from C:\Users\hp\OneDrive\Desktop\yes: 100%|█████████████████████████| 155/155 [00:00<00:00, 356.41it/s]
Loading images from C:\Users\hp\OneDrive\Desktop\no: 100%|████████████████████████████| 98/98 [00:00<00:00, 333.93it/s]



Processing tumor images...


100%|████████████████████████████████████████████████████████████████████████████████| 155/155 [04:28<00:00,  1.73s/it]



Processing normal images...


100%|██████████████████████████████████████████████████████████████████████████████████| 98/98 [01:59<00:00,  1.22s/it]


Segmentation metrics saved to segmentation_results\segmentation_metrics.csv

Average metrics:
        accuracy       iou      dice
type                                
normal  0.922796  0.000000  0.000000
tumor   0.935994  0.608187  0.734806
